In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.collab import *
from fastai.tabular import *

Levanto el dataset de movielens que ya tengo bajado

In [3]:
path = Path('/home/fvadell/trabajo/recs/ml-latest-small')
path.ls()

[PosixPath('/home/fvadell/trabajo/recs/ml-latest-small/links.csv'),
 PosixPath('/home/fvadell/trabajo/recs/ml-latest-small/README.txt'),
 PosixPath('/home/fvadell/trabajo/recs/ml-latest-small/ratings.csv'),
 PosixPath('/home/fvadell/trabajo/recs/ml-latest-small/movies.csv'),
 PosixPath('/home/fvadell/trabajo/recs/ml-latest-small/tags.csv')]

In [4]:
movies = pd.read_csv(path/'movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tags = pd.read_csv(path/'links.csv')

In [6]:
tags.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Acá se ve la matriz de interacciones, presentada como un csv. Agrego copias de algunas columnas pero con otro data type por si necesito

In [7]:
ratings = pd.read_csv(path/'ratings.csv')
ratings['y'] = ratings.rating.apply(str)
ratings['user'] = ratings['userId'].apply(lambda x:str(int(x))).astype('category')
ratings['item'] = ratings['movieId'].apply(lambda x:str(int(x))).astype('category')
ratings.head()

,userId,movieId,rating,timestamp,y,user,item
0,1,1,4.0,964982703,4.0,1,1
1,1,3,4.0,964981247,4.0,1,3
2,1,6,4.0,964982224,4.0,1,6
3,1,47,5.0,964983815,5.0,1,47
4,1,50,5.0,964982931,5.0,1,50


Lo junto con el data frame movies para tener también el nombre de la película y sus tag de género

In [8]:
ratings_con_nombres = pd.merge(ratings, movies, on = 'movieId')

In [9]:
ratings_con_nombres.head()

,userId,movieId,rating,timestamp,y,user,item,title,genres
0,1,1,4.0,964982703,4.0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,4.0,5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,4.5,7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,2.5,15,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,4.5,17,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Armo el data bunch de fastai con la matriz de interacciones. Le pedimos que arme un validation set con el 15% de los datos y que use un batchsize de 100.

In [10]:
data = CollabDataBunch.from_df(ratings[['userId','movieId','rating']], valid_pct=0.15, bs=100)

El modelo que vamos a utilizar usa Embeddings para los usuarios y para las películas. Hay que especificarle el tamaño de los embeddings que vamos a usar (100 en ambos casos) y cuántas capas lineales y de qué tamaño.

In [11]:
emb_szs = {
  "userId": 100,
  "movieId": 100,
}

layers = [50, 10, 10]

In [12]:
u,m = data.train_ds.x.classes.values()


In [13]:
y_range = [0, 5.5]

In [14]:
learn_small = collab_learner(data, emb_szs=emb_szs, 
                             use_nn = True, emb_drop= .3, 
                             layers = [50, 10, 10], 
                             y_range=y_range)

In [15]:
learn_small.fit_one_cycle(3, 6.31E-02)

epoch,train_loss,valid_loss,time
0,0.823577,0.805387,00:15
1,0.751112,0.753496,00:26
2,0.664774,0.727286,00:22


learn_small.model contiene el modelo entero. Si le paso un tensor con los usuarios y películas va a llamar al forward del modelo y me va a devolver la predicción de la red.

In [16]:
learn_small.model(torch.LongTensor([0]),torch.LongTensor([0]))

tensor([[3.1404]], grad_fn=<AddBackward0>)

In [17]:
torch.nn.Embedding??

In [18]:
# Esto me muestra todos los objetos que tiene definido el modelo
# NO significa que se vayan a usar todos.
# NO significa que no se use algo más
# Para saber exactamente cómo va a ser el flujo de datos tengo que ver el forward
learn_small.model

EmbeddingNN(
  (embeds): ModuleList(
    (0): Embedding(611, 100)
    (1): Embedding(9157, 100)
  )
  (emb_drop): Dropout(p=0.3, inplace=False)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=200, out_features=50, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=50, out_features=10, bias=True)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): ReLU(inplace=True)
    (8): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [19]:
# Para ver el forward veo primero el código de collab_learner para ver qué modelo usa
# Veo el código de collab_learner
# >> collab_learner??
# Se ve que crea un modelo de tipo EmbeddingNN
#if use_nn: model = EmbeddingNN(emb_szs=emb_szs, layers=layers, ps=ps, emb_drop=emb_drop, y_range=y_range,
#                                   use_bn=use_bn, bn_final=bn_final, **learn_kwargs)

In [20]:
# Veo entonces si el modelo EmbeddingNN tiene definido el forward
# >> EmbeddingNN??
# Se ve que tiene definido un forward pero que llama al forward de su super que es TabularModel
#def forward(self, users:LongTensor, items:LongTensor) -> Tensor:
#        return super().forward(torch.stack([users,items], dim=1), None)

In [21]:
# Veo si en TabularModel hay definido un forward
# >> TabularModel??
# Se ve que si existe un y_range entonces el resultado lo pasa por una sigmoid
# pero si no tene y_range simplemente devuelve el resultado de self.layers(x)
# es decir pasar el input directamente por todas las capas que tenga definidas el modelo
#def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
#        if self.n_emb != 0:
#            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
#            x = torch.cat(x, 1)
#            x = self.emb_drop(x)
#        if self.n_cont != 0:
#            x_cont = self.bn_cont(x_cont)
#            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
#        x = self.layers(x)
#        if self.y_range is not None:
#            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
#        return x

Voy a hacer la prueba de tomar los embeddings de la primer película y el primer usuario y pasarlos "a mano" por el resto de la red neuronal. Para eso tengo en cuenta la estructura del modelo.

Voy a pasar el embedding por:

emb_drop

bn_cont no lo uso porque está seteado con la primer variable en 0 (ver)

layers

sigmoid

In [22]:
# u y a son los embeddings para el primer usuario y la primer pelicula respectivamente
u = learn_small.model.embeds[0](torch.LongTensor([0]))
a = learn_small.model.embeds[1](torch.LongTensor([0]))

In [23]:
u, a

(tensor([[ 0.0034,  0.0032,  0.0030, -0.0034,  0.0023, -0.0010,  0.0005, -0.0020,
          -0.0045, -0.0061,  0.0019, -0.0022, -0.0001, -0.0024,  0.0045,  0.0062,
           0.0034, -0.0080, -0.0015, -0.0012, -0.0002, -0.0039,  0.0010,  0.0017,
           0.0055, -0.0074,  0.0043, -0.0033,  0.0070, -0.0057, -0.0015,  0.0006,
          -0.0016, -0.0009, -0.0008,  0.0001,  0.0018,  0.0010,  0.0025, -0.0081,
           0.0051,  0.0024, -0.0008, -0.0012, -0.0086,  0.0017,  0.0017,  0.0015,
           0.0023,  0.0040,  0.0046, -0.0072, -0.0020, -0.0005,  0.0023, -0.0011,
           0.0017,  0.0038, -0.0004,  0.0023,  0.0046,  0.0038, -0.0005,  0.0018,
          -0.0004, -0.0036, -0.0004, -0.0034, -0.0012, -0.0035,  0.0018, -0.0042,
          -0.0046,  0.0052,  0.0008, -0.0035,  0.0003, -0.0028, -0.0005,  0.0017,
           0.0019, -0.0048, -0.0009,  0.0048,  0.0017,  0.0008, -0.0046, -0.0008,
          -0.0023, -0.0002,  0.0011, -0.0062,  0.0030, -0.0027, -0.0015, -0.0016,
           0.002

Junto los embeddings

In [24]:
x = torch.cat([u,a],dim=1)

In [25]:
learn_small.model.layers

Sequential(
  (0): Linear(in_features=200, out_features=50, bias=True)
  (1): ReLU(inplace=True)
  (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=50, out_features=10, bias=True)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): ReLU(inplace=True)
  (8): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=10, out_features=1, bias=True)
)

In [26]:
# Lo paso por la capa de emb_drop, layers y después aplico la sigmoid.
# Lo que consigo es la predicción
x = learn_small.model.emb_drop(x)
x = learn_small.model.layers(x)
x = (y_range[1]-y_range[0]) * torch.sigmoid(x) + y_range[0]
x

tensor([[3.1404]], grad_fn=<AddBackward0>)

tomo el usuario con userId = 1

In [27]:
u = learn_small.model.embeds[0](torch.LongTensor([0]))

In [28]:
u.shape

torch.Size([1, 100])

Y la matriz entera de embeddings de películas

In [29]:
emb_m = learn_small.model.embeds[1].weight

In [30]:
emb_m

Parameter containing:
tensor([[-6.6601e-04,  8.1706e-03, -3.4111e-03,  ..., -5.8973e-03,
         -3.9851e-03, -3.5983e-03],
        [ 7.2922e-01, -3.9186e-01,  3.6558e-01,  ...,  3.1164e-01,
          2.9151e-01,  3.2152e-01],
        [-7.3079e-01,  7.5796e-01,  1.1824e-01,  ...,  1.6944e-02,
         -1.2032e+00,  4.3465e-01],
        ...,
        [ 3.3418e-01, -1.9305e-02,  2.6836e-02,  ...,  2.5326e-01,
         -6.3755e-01, -1.0193e-02],
        [ 3.6763e-01,  1.7498e-01, -6.1660e-03,  ..., -3.7421e-01,
          4.1561e-01, -2.2166e-01],
        [ 2.6976e-02,  3.5013e-01,  4.6290e-01,  ...,  3.9850e-01,
         -3.1122e-03,  3.8838e-02]], requires_grad=True)

In [31]:
emb_m.shape

torch.Size([9157, 100])

In [32]:
emb_m.shape[0]

9157

In [33]:
#Si queremos hacer una prueba de rapidez aumentando por 100 la cantidad de películas podemos correr esto
#emb_m = torch.cat([emb_m]*100)

In [34]:
u[0]

tensor([ 0.0034,  0.0032,  0.0030, -0.0034,  0.0023, -0.0010,  0.0005, -0.0020,
        -0.0045, -0.0061,  0.0019, -0.0022, -0.0001, -0.0024,  0.0045,  0.0062,
         0.0034, -0.0080, -0.0015, -0.0012, -0.0002, -0.0039,  0.0010,  0.0017,
         0.0055, -0.0074,  0.0043, -0.0033,  0.0070, -0.0057, -0.0015,  0.0006,
        -0.0016, -0.0009, -0.0008,  0.0001,  0.0018,  0.0010,  0.0025, -0.0081,
         0.0051,  0.0024, -0.0008, -0.0012, -0.0086,  0.0017,  0.0017,  0.0015,
         0.0023,  0.0040,  0.0046, -0.0072, -0.0020, -0.0005,  0.0023, -0.0011,
         0.0017,  0.0038, -0.0004,  0.0023,  0.0046,  0.0038, -0.0005,  0.0018,
        -0.0004, -0.0036, -0.0004, -0.0034, -0.0012, -0.0035,  0.0018, -0.0042,
        -0.0046,  0.0052,  0.0008, -0.0035,  0.0003, -0.0028, -0.0005,  0.0017,
         0.0019, -0.0048, -0.0009,  0.0048,  0.0017,  0.0008, -0.0046, -0.0008,
        -0.0023, -0.0002,  0.0011, -0.0062,  0.0030, -0.0027, -0.0015, -0.0016,
         0.0021,  0.0018, -0.0062, -0.00

Creo una matriz del mismo tamaño que la matriz de embeddings pero que en cada fila tenga el embeddings del usuario para el que quiero la recomendación

In [35]:
u = u[0].repeat(emb_m.shape[0]).reshape(emb_m.shape[0], -1)

In [36]:
u = u.float()

In [37]:
u[0]

tensor([ 0.0034,  0.0032,  0.0030, -0.0034,  0.0023, -0.0010,  0.0005, -0.0020,
        -0.0045, -0.0061,  0.0019, -0.0022, -0.0001, -0.0024,  0.0045,  0.0062,
         0.0034, -0.0080, -0.0015, -0.0012, -0.0002, -0.0039,  0.0010,  0.0017,
         0.0055, -0.0074,  0.0043, -0.0033,  0.0070, -0.0057, -0.0015,  0.0006,
        -0.0016, -0.0009, -0.0008,  0.0001,  0.0018,  0.0010,  0.0025, -0.0081,
         0.0051,  0.0024, -0.0008, -0.0012, -0.0086,  0.0017,  0.0017,  0.0015,
         0.0023,  0.0040,  0.0046, -0.0072, -0.0020, -0.0005,  0.0023, -0.0011,
         0.0017,  0.0038, -0.0004,  0.0023,  0.0046,  0.0038, -0.0005,  0.0018,
        -0.0004, -0.0036, -0.0004, -0.0034, -0.0012, -0.0035,  0.0018, -0.0042,
        -0.0046,  0.0052,  0.0008, -0.0035,  0.0003, -0.0028, -0.0005,  0.0017,
         0.0019, -0.0048, -0.0009,  0.0048,  0.0017,  0.0008, -0.0046, -0.0008,
        -0.0023, -0.0002,  0.0011, -0.0062,  0.0030, -0.0027, -0.0015, -0.0016,
         0.0021,  0.0018, -0.0062, -0.00

In [38]:
emb_m.shape

torch.Size([9157, 100])

Concateno cada vector del embedding de movies con el del usuario

In [39]:
x = torch.cat([u,emb_m],dim=1)

In [40]:
x.shape

torch.Size([9157, 200])

Lo paso por la capa de emb_drop, layers y despues aplico la sigmoid. Lo que consigo es la predicción del rating de ese usuario para cada una de las películas

In [41]:
x.shape

torch.Size([9157, 200])

In [42]:
# Lo paso por la capa de emb_drop, layers y después aplico la sigmoid.
# Lo que consigo es la predicción
x = learn_small.model.emb_drop(x)
x = learn_small.model.layers(x)
x = (y_range[1]-y_range[0]) * torch.sigmoid(x) + y_range[0]
x

tensor([[3.1404],
        [3.8452],
        [3.3335],
        ...,
        [3.2563],
        [3.6222],
        [3.2587]], grad_fn=<AddBackward0>)

In [43]:
learn_small.data.dataset.x.classes

OrderedDict([('userId',
              array(['#na#', '1', '2', '3', ..., '607', '608', '609', '610'], dtype='<U21')),
             ('movieId',
              array(['#na#', '1', '2', '3', ..., '193581', '193583', '193585', '193587'], dtype='<U21'))])

Busco los ids de películas en el dataset

In [44]:
PELIS = learn_small.data.dataset.x.classes['movieId']

In [45]:
PELIS

array(['#na#', '1', '2', '3', ..., '193581', '193583', '193585', '193587'], dtype='<U21')

In [46]:
len(PELIS)

9157

Le agrego a cada película su rating y armo un ranking de las que mejor rating predicho tienen.

In [47]:
# Armo el top de las pelis con mayor predicción de ranking
top = sorted([(PELIS[i],round(n[0],2) ) for i,n in enumerate(x.tolist())],key=lambda x:x[1],reverse=True )
top[:10]

[('318', 4.38),
 ('1136', 4.3),
 ('3451', 4.29),
 ('527', 4.27),
 ('720', 4.27),
 ('1248', 4.27),
 ('1104', 4.26),
 ('4973', 4.26),
 ('8132', 4.26),
 ('750', 4.25)]

In [48]:
top_df = pd.DataFrame(top)

In [49]:
top_df.columns = ['movieId', 'prediction']

In [50]:
top_df['movieId']=top_df['movieId'].astype(str)

In [51]:
movies['movieId']=movies['movieId'].astype(str)

In [52]:
top_df = pd.merge(top_df, movies, on = 'movieId')

In [53]:
top_df.head()

,movieId,prediction,title,genres
0,318,4.38,"Shawshank Redemption, The (1994)",Crime|Drama
1,1136,4.30,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
2,3451,4.29,Guess Who's Coming to Dinner (1967),Drama
3,527,4.27,Schindler's List (1993),Drama|War
4,720,4.27,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy


In [54]:
ratings_con_nombres['movieId']=ratings_con_nombres['movieId'].astype(str)

In [55]:
movies_for_user = ratings_con_nombres[ratings_con_nombres['userId']==1]

In [56]:
movies_for_user['movieId']=movies_for_user['movieId'].astype(str)

/home/fvadell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
a = pd.merge(movies_for_user, top_df, on = 'movieId', how = 'outer').sort_values(by = 'prediction', ascending = False).reset_index(drop = True)

In [58]:
ratings_con_nombres[ratings_con_nombres['userId']==1].sort_values(by = 'rating', ascending = False)

,userId,movieId,rating,timestamp,y,user,item,title,genres
16250,1,5060,5.0,964984002,5.0,1,5060,M*A*S*H (a.k.a. MASH) (1970),Comedy|Drama|War
14053,1,2872,5.0,964981680,5.0,1,2872,Excalibur (1981),Adventure|Fantasy
9066,1,1291,5.0,964981909,5.0,1,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
9206,1,1298,5.0,964984086,5.0,1,1298,Pink Floyd: The Wall (1982),Drama|Musical
14254,1,2948,5.0,964982191,5.0,1,2948,From Russia with Love (1963),Action|Adventure|Thriller
14201,1,2947,5.0,964982176,5.0,1,2947,Goldfinger (1964),Action|Adventure|Thriller
14169,1,2944,5.0,964981872,5.0,1,2944,"Dirty Dozen, The (1967)",Action|Drama|War
14078,1,2899,5.0,964982703,5.0,1,2899,Gulliver's Travels (1939),Adventure|Animation|Children
13849,1,2858,5.0,964980868,5.0,1,2858,American Beauty (1999),Drama|Romance
13497,1,2700,5.0,964980985,5.0,1,2700,"South Park: Bigger, Longer and Uncut (1999)",Animation|Comedy|Musical


#### Ranking de recomendaciones para el usuario.

Los valores que tienen NaN en user y en rating se refieren a películas que el usuario no vio

In [59]:
a.filter(['userId', 'movieId', 'rating', 'prediction', 'title_y', 'genres_y'])

,userId,movieId,rating,prediction,title_y,genres_y
0,NaN,318,NaN,4.38,"Shawshank Redemption, The (1994)",Crime|Drama
1,1.0,1136,5.0,4.30,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
2,NaN,3451,NaN,4.29,Guess Who's Coming to Dinner (1967),Drama
3,NaN,720,NaN,4.27,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
4,NaN,1248,NaN,4.27,Touch of Evil (1958),Crime|Film-Noir|Thriller
5,1.0,527,5.0,4.27,Schindler's List (1993),Drama|War
6,NaN,1104,NaN,4.26,"Streetcar Named Desire, A (1951)",Drama
7,NaN,4973,NaN,4.26,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance
8,NaN,8132,NaN,4.26,Gladiator (1992),Action|Drama
9,NaN,750,NaN,4.25,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War


Otra forma de reordenar los datos es usando el topk que viene con torch

In [60]:
b,a = torch.topk(x.view(-1),10)
[(PELIS[i],round(n,2) ) for i,n in zip(a.tolist(),b.tolist()) ]

[('318', 4.38),
 ('1136', 4.3),
 ('3451', 4.29),
 ('527', 4.27),
 ('1248', 4.27),
 ('720', 4.27),
 ('1104', 4.26),
 ('4973', 4.26),
 ('8132', 4.26),
 ('905', 4.25)]

Ejemplo de module de pytorch que toma dos embeddings de avisos de casas y un embedding de avisos de usuario y los pasa por una capa lineal

In [61]:
class MiMo(nn.Module):     
    def __init__(self, D_in=15, H=5, D_out=10):
        
        super(MiMo, self).__init__()
        self.casa1 = learn_small.model.embeds[0]
        self.casa2 = learn_small.model.embeds[0]
        self.usuario = learn_small.model.embeds[1]
        self.densa = nn.Linear(300,1)
    
    def forward(self, x):
        # Se asume que la estructura del input de entrada (x) es un tensor de vectores de (al menos) 3 coordenadas
        # La primer y segunda coordenadas son ids de casa y la tercera es un id de usuario
        x = torch.cat([self.casa1(x[:,0]),self.casa2(x[:,1]),self.usuario(x[:,2])],dim=1)        
        #x = torch.cat([x,features])   #Puedo pedir que agregue features al embedding antes de pasarlo por la capa lineal
        return self.densa(x)
        #return 1

In [62]:
torch.LongTensor([[0,1,2], [3,4,5]])[:,0]

tensor([0, 3])

In [63]:
x = MiMo()
x(torch.LongTensor([[0,1,2]]))

tensor([[-0.8499]], grad_fn=<AddmmBackward>)

# Pytorch NN

#### Aca hay algunas pruebas con una red de pytorch

In [64]:
import torch
import torch.nn as nn

In [65]:
# Defining input size, hidden layer size, output size and batch size respectively
n_in, n_h, n_out, batch_size = 10, 5, 1, 10

In [66]:
# Create dummy input and target tensors (data)
# Creo un tensor formado por 10 (batch_size) vectores de tamaño 10 (n_in)
x = torch.randn(batch_size, n_in)

In [67]:
# Y creo la salida esperada para cada uno de esos vectores
y = torch.tensor([[1.0], [0.0], [0.0], 
[1.0], [1.0], [1.0], [0.0], [0.0], [1.0], [1.0]])

In [68]:
# Create a model
# nn.Sequential crea una sequencia de modulos (o capas). En este caso armamos una que tenga
# Una capa lineal, una Relu, una lineal y una Sigmoid
model = nn.Sequential(nn.Linear(n_in, n_h),
   nn.ReLU(),
   nn.Linear(n_h, n_out),
   nn.Sigmoid())

In [69]:
model

Sequential(
  (0): Linear(in_features=10, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
  (3): Sigmoid()
)

In [70]:
# Defino la función de pérdida
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [71]:
# Gradient Descent
for epoch in range(50):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(x)

   # Compute and print loss
   loss = criterion(y_pred, y)
   #print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

print(y_pred)
print(y)

tensor([[0.4726],
        [0.5719],
        [0.5268],
        [0.4677],
        [0.4646],
        [0.4554],
        [0.5081],
        [0.4607],
        [0.4568],
        [0.4714]], grad_fn=<SigmoidBackward>)
tensor([[1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [72]:
import os
import numpy as np # linear algebra
import pandas as pd 
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SparseAdam,Adam,Adagrad,SGD
#from livelossplot import PlotLosses

In [73]:
path = '/home/fvadell/trabajo/recs/ml-latest-small'

In [74]:
ratings = pd.read_csv(path+'/ratings.csv')
ratings['y'] = ratings.rating.apply(str)
ratings['user'] = ratings['userId'].apply(lambda x:str(int(x))).astype('category')
ratings['item'] = ratings['movieId'].apply(lambda x:str(int(x))).astype('category')
ratings.head()

,userId,movieId,rating,timestamp,y,user,item
0,1,1,4.0,964982703,4.0,1,1
1,1,3,4.0,964981247,4.0,1,3
2,1,6,4.0,964982224,4.0,1,6
3,1,47,5.0,964983815,5.0,1,47
4,1,50,5.0,964982931,5.0,1,50


In [75]:
# Creo train y test sets
train, test = train_test_split(ratings, test_size=0.2)

In [76]:
test.head()

,userId,movieId,rating,timestamp,y,user,item
41510,282,1240,4.5,1378490530,4.5,282,1240
26006,181,169,3.0,845470321,3.0,181,169
28716,199,1635,3.0,946223222,3.0,199,1635
60211,387,46967,3.0,1175837057,3.0,387,46967
17660,111,79428,3.0,1516155080,3.0,111,79428


In [77]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(100, 30)
        self.fc2 = nn.Linear(30, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=100, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
)


In [78]:
x = torch.randn(5,100)

In [79]:
x

tensor([[ 4.5609e-01, -4.6341e-01, -9.6664e-02,  6.7768e-01,  2.0181e-01,
          1.3753e+00, -6.8645e-01,  4.3263e-01,  8.7143e-01, -1.3928e+00,
         -7.5869e-02,  3.0057e-01, -1.1437e-01,  9.1657e-01,  3.6378e-01,
         -1.4751e-01, -1.0081e+00, -4.5702e-01,  2.8071e-01, -1.8049e-01,
         -8.0234e-01,  1.5567e-02, -1.1482e-01,  6.4846e-01, -1.6348e+00,
          2.1345e-01,  9.1618e-01, -4.1298e-02,  1.0860e+00, -1.0774e+00,
          3.6840e-01, -3.6696e-01, -7.2697e-01,  3.9231e-01,  2.2491e+00,
         -1.0856e+00,  8.4576e-01, -8.6410e-01, -2.3594e+00,  4.5332e-01,
         -7.5271e-01, -1.0927e-01,  1.0849e+00, -8.9218e-01,  5.2379e-01,
         -5.2717e-01, -1.1673e+00, -7.4381e-03,  9.4621e-01,  1.1812e+00,
          1.3772e+00, -1.2868e-01, -2.7945e-01,  8.8720e-01,  1.7039e+00,
          2.0543e-01,  9.7300e-01,  1.0380e+00, -1.6734e-01,  6.0135e-01,
         -1.2684e+00, -2.7601e-01,  1.5678e+00, -1.2606e+00, -9.2418e-01,
          6.2502e-01, -1.4790e+00,  6.

In [80]:
net = Net()

In [81]:
x = torch.Tensor(x)

In [82]:
x.shape

torch.Size([5, 100])

In [83]:
net.forward(x)

tensor([[0.2414],
        [0.2684],
        [0.1639],
        [0.2359],
        [0.2891]], grad_fn=<AddmmBackward>)

In [84]:
params = list(net.parameters())

In [85]:
# Hay 2 sets de parámetros por cada capa porque las capas Lineales tienen tanto Weights como Bias.
# Si seteara bias=False cuando se llama a nn.Linear entonces tendría 3 sets de parámetros
len(params)

6

In [86]:
model = nn.Sequential(Net())

In [87]:
model

Sequential(
  (0): Net(
    (fc1): Linear(in_features=100, out_features=30, bias=True)
    (fc2): Linear(in_features=30, out_features=50, bias=True)
    (fc3): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [88]:
# Defino la función de pérdida
criterion = torch.nn.MSELoss()
# Construct the optimizer (Stochastic Gradient Descent in this case)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [89]:
x

tensor([[ 4.5609e-01, -4.6341e-01, -9.6664e-02,  6.7768e-01,  2.0181e-01,
          1.3753e+00, -6.8645e-01,  4.3263e-01,  8.7143e-01, -1.3928e+00,
         -7.5869e-02,  3.0057e-01, -1.1437e-01,  9.1657e-01,  3.6378e-01,
         -1.4751e-01, -1.0081e+00, -4.5702e-01,  2.8071e-01, -1.8049e-01,
         -8.0234e-01,  1.5567e-02, -1.1482e-01,  6.4846e-01, -1.6348e+00,
          2.1345e-01,  9.1618e-01, -4.1298e-02,  1.0860e+00, -1.0774e+00,
          3.6840e-01, -3.6696e-01, -7.2697e-01,  3.9231e-01,  2.2491e+00,
         -1.0856e+00,  8.4576e-01, -8.6410e-01, -2.3594e+00,  4.5332e-01,
         -7.5271e-01, -1.0927e-01,  1.0849e+00, -8.9218e-01,  5.2379e-01,
         -5.2717e-01, -1.1673e+00, -7.4381e-03,  9.4621e-01,  1.1812e+00,
          1.3772e+00, -1.2868e-01, -2.7945e-01,  8.8720e-01,  1.7039e+00,
          2.0543e-01,  9.7300e-01,  1.0380e+00, -1.6734e-01,  6.0135e-01,
         -1.2684e+00, -2.7601e-01,  1.5678e+00, -1.2606e+00, -9.2418e-01,
          6.2502e-01, -1.4790e+00,  6.

In [90]:
model(x).shape

torch.Size([5, 1])

In [91]:
y = torch.Tensor([1,0.2,0.7,0.1,0])

In [92]:
y.shape

torch.Size([5])

In [93]:
y.resize_((5,1))
y.shape

torch.Size([5, 1])

In [94]:
# Gradient Descent
for epoch in range(50):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(x)

   # Compute and print loss
   loss = criterion(y_pred, y)
   #print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

print(y_pred)
print(y)

tensor([[0.8860],
        [0.2888],
        [0.6206],
        [0.2439],
        [0.1371]], grad_fn=<AddmmBackward>)
tensor([[1.0000],
        [0.2000],
        [0.7000],
        [0.1000],
        [0.0000]])
